In [1]:
import pandas as pd

In [2]:
ruta="/home/donsson/proyectos/VENTAS/Referencias Promo Autoparts.xlsx"
facturas = "/home/donsson/proyectos/API/ventashistoricas3semanas.parquet"

promos = pd.read_excel(ruta)

df_f = pd.read_parquet(facturas)

In [13]:
promos["Referencia interna"].count()

np.int64(143)

In [16]:
df_f["Referencia interna"] = df_f["product_name"].str.extract(r"\[([^\]]+)\]")
df_f["Nombre"] = df_f["product_name"].str.replace(r"\[.*?\]\s*", "", regex=True)


In [5]:
df_f["ref"] = df_f["Referencia interna"].astype(str).str.strip().str.upper()
promos["ref"] = promos["Referencia interna"].astype(str).str.strip().str.upper()

# Hacemos inner join: solo los que están en la promo
df_promo_facturas = promos.merge(
    df_f,
    how="inner",
    on="ref",
    suffixes=("_promo", "_factura")
)

In [17]:
df_promo_facturas.head(10)

,Referencia interna_promo,Nombre_promo,Categoría interna,Clase,Precio de venta,Cantidad a mano,ref,price_subtotal,company_id,id,quantity,product_id_num,product_name,invoice_id_num,invoice_name,date_invoice,Referencia interna_factura,Nombre_factura
0,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS Y MASTER,Industrias Donsson / Filtros / Autoparts / Imp...,C,18600,104,DLE00909189,224000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1610871,20.0,50107,[DLE00909189] G909 FILTRO ACEITE - RENAULT KOL...,436854,FMDE3839,2025-09-10 16:20:11,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS DIESEL (18...
1,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS Y MASTER,Industrias Donsson / Filtros / Autoparts / Imp...,C,18600,104,DLE00909189,37800.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1607856,3.0,50107,[DLE00909189] G909 FILTRO ACEITE - RENAULT KOL...,436077,PV7E8110 Mostrador Bucaramanga T2/2433,2025-09-05 17:52:28,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS DIESEL (18...
2,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS Y MASTER,Industrias Donsson / Filtros / Autoparts / Imp...,C,18600,104,DLE00909189,11200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1599824,1.0,50107,[DLE00909189] G909 FILTRO ACEITE - RENAULT KOL...,433903,FCAL5335,2025-08-26 14:00:55,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS DIESEL (18...
3,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS Y MASTER,Industrias Donsson / Filtros / Autoparts / Imp...,C,18600,104,DLE00909189,25200.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1597698,2.0,50107,[DLE00909189] G909 FILTRO ACEITE - RENAULT KOL...,433337,PV3E19524 Mostrador Valladolid T1/22962,2025-08-22 17:17:11,DLE00909189,G909 FILTRO ACEITE - RENAULT KOLEOS DIESEL (18...
4,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,90000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1608954,2.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,436375,FCOT26067,2025-09-08 17:07:18,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)
5,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,51000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1604585,1.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,435274,PV3E19639 Mostrador Valladolid T2/15138,2025-09-01 21:51:25,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)
6,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,51000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1602295,1.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,434570,FCAL5384,2025-08-28 21:32:58,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)
7,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,510000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1597146,10.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,433201,PV1E10865 Most Cota14059,2025-08-21 21:29:55,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)
8,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,51000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1596826,1.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,433113,PVE53806 Mostrador Medellín T2/5947,2025-08-21 18:55:42,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)
9,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO,Industrias Donsson / Filtros / Autoparts / Imp...,C,75000,105,DCS00740189,102000.0,"[1, 'INDUSTRIAS DONSSON S.A.S.']",1596727,2.0,49948,[DCS00740189] GS740 FILTRO COMBUSTIBLE - HINO ...,433075,PV7E7971 Mostrador Bucaramanga T2/2310,2025-08-21 17:05:33,DCS00740189,GS740 FILTRO COMBUSTIBLE - HINO (189 GS740)


In [12]:

df_promo_facturas_limpio = df_promo_facturas[[
    "ref",
    "Clase",
    "Nombre_promo",
    "date_invoice",
    "quantity",
    "price_subtotal"
]].copy()

# ✅ Paso 2. Convertimos a datetime (sin warnings)
df_promo_facturas_limpio.loc[:, "date_invoice"] = pd.to_datetime(df_promo_facturas_limpio["date_invoice"])

# ✅ Paso 3. Creamos columna Fecha solo con año-mes-día
df_promo_facturas_limpio["Fecha"] = df_promo_facturas_limpio["date_invoice"].dt.date

# ✅ Paso 4. Filtrar septiembre
inicio_promo = pd.to_datetime("2025-09-01")
fin_promo    = pd.to_datetime("2025-09-30")

df_promo_facturas_limpio = df_promo_facturas_limpio[
    (df_promo_facturas_limpio["date_invoice"] >= inicio_promo) &
    (df_promo_facturas_limpio["date_invoice"] <= fin_promo)
]

# ✅ Paso 5. Agrupar por día + producto
df_resumen_dia = (df_promo_facturas_limpio
                  .groupby(["ref","Clase", "Nombre_promo", "Fecha"], as_index=False)
                  .agg({
                      "quantity": "sum",
                      "price_subtotal": "sum"
                  }))


df_resumen_dia.to_excel("promoseptiembreautoparts.xlsx")

